In [ ]:
# -*- coding: utf-8 -*-
"""
Train (3 keypoints: left back paw, right back paw, tail set)
- NORMALIZE_MODE = "zscore" (train1相当) or "tail_minmax" (train2相当)
- outputs:
  - data/train/fig/curve_<TS>.png
  - data/train/val_misclassified/val_misclassified_<TS>.csv
  - data/train/train1_model or train2_model / ivdd_lstm_3kp_<TS>_best.keras / _final.keras
"""

import os
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "-1")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")

import re, glob
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

print("TensorFlow:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

# ====== パス設定（あなたのリポジトリ最上位に合わせて修正可）======
REPO_ROOT = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master"
TRAIN_ROOT = os.path.join(REPO_ROOT, "data", "train")
TEST_ROOT  = os.path.join(REPO_ROOT, "data", "test")

TRAIN_CSV_DIR = os.path.join(TRAIN_ROOT, "train_csv")
TRAIN_GLOB    = os.path.join(TRAIN_CSV_DIR, "*.csv")
FIG_DIR       = os.path.join(TRAIN_ROOT, "fig")
VALERR_DIR    = os.path.join(TRAIN_ROOT, "val_misclassified")
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(VALERR_DIR, exist_ok=True)

# ====== 実行設定 ======
RUN_ID = datetime.now().strftime("%Y%m%d-%H%M%S")

# 正規化選択: "zscore" (train1相当) / "tail_minmax" (train2相当)
NORMALIZE_MODE = "tail_minmax"   # ←必要に応じて "zscore" に変更

# モデル保存先はモードに応じて
MODEL_DIR = os.path.join(TRAIN_ROOT, "train1_model" if NORMALIZE_MODE=="zscore" else "train2_model")
os.makedirs(MODEL_DIR, exist_ok=True)

# ====== データ設定 ======
KEYPOINTS = ["left back paw", "right back paw", "tail set"]
USE_LIKELIHOOD      = False
MIN_KEEP_LIKELIHOOD = 0.6
SEQ_LEN  = 60
STRIDE   = 30
DIMS     = 6  # 3keypoints × (x,y)

# ====== 学習ハイパラ ======
N_HIDDEN   = 30
BATCH_SIZE = 30
EPOCHS     = 60
LR         = 1e-4
L2_LAMBDA  = 1e-4

VAL_SPLIT_BY_FILE = True

# ラベル: 0=normal, 1=ivdd
CLASS_NAMES  = ["normal", "ivdd"]
NAME2IDX     = {"normal":0, "ivdd":1}

# ====== ユーティリティ ======
def _norm_name(s: str) -> str:
    return "".join(ch for ch in s.lower() if ch not in " _-")

def _resolve_keypoints(all_bodyparts, requested):
    norm2orig = {}
    for bp in all_bodyparts:
        k = _norm_name(bp)
        if k not in norm2orig:
            norm2orig[k] = bp
    resolved, missing = [], []
    for req in requested:
        k = _norm_name(req)
        if k in norm2orig:
            resolved.append(norm2orig[k])
        else:
            missing.append(req)
    if missing:
        raise ValueError(f"指定KPが見つかりません: {missing}\n利用可能: {all_bodyparts}")
    return resolved

def infer_label_from_filename(path: str) -> int:
    """
    'ivdd', 'ivdd1','ivdd2',... を ivdd と判定。'normal' は normal。
    ファイル名→未決なら先頭トークン→親ディレクトリの順で決める。
    """
    name = os.path.basename(path).lower()
    stem = os.path.splitext(name)[0]
    tokens = [t for t in re.split(r'[^a-z0-9]+', stem) if t]
    token_set = set(tokens)

    has_ivdd = any(t == "ivdd" or t.startswith("ivdd") for t in tokens)
    has_normal = "normal" in token_set

    if has_ivdd and not has_normal:
        return NAME2IDX["ivdd"]
    if has_normal and not has_ivdd:
        return NAME2IDX["normal"]

    if tokens and tokens[0] in NAME2IDX:
        return NAME2IDX[tokens[0]]

    parent_tokens = [t for t in re.split(r'[^a-z0-9]+', os.path.dirname(path).lower()) if t]
    p_has_ivdd   = any(t == "ivdd" or t.startswith("ivdd") for t in parent_tokens)
    p_has_normal = "normal" in set(parent_tokens)
    if p_has_ivdd and not p_has_normal:
        return NAME2IDX["ivdd"]
    if p_has_normal and not p_has_ivdd:
        return NAME2IDX["normal"]

    raise ValueError(f"ラベル不明: {name}（'ivdd' か 'normal' を含めてください）")

def read_dlc_3kp_xy(csv_path: str, keypoints, use_likelihood=True, min_keep_likelihood=0.6):
    df = pd.read_csv(csv_path, header=[0,1,2], index_col=0)
    bodyparts = list({bp for (_, bp, _) in df.columns})
    use_kps = _resolve_keypoints(bodyparts, keypoints)

    cols = {}
    for bp in use_kps:
        cols[f"{bp}_x"] = df.xs((bp, "x"), level=[1,2], axis=1)
        cols[f"{bp}_y"] = df.xs((bp, "y"), level=[1,2], axis=1)
    X_df = pd.concat(cols.values(), axis=1)
    X_df.columns = list(cols.keys())

    if use_likelihood:
        for bp in use_kps:
            try:
                lcol = df.xs((bp, "likelihood"), level=[1,2], axis=1).values.flatten()
                low = lcol < min_keep_likelihood
                for c in [f"{bp}_x", f"{bp}_y"]:
                    v = X_df[c].values
                    v[low] = np.nan
                    X_df[c] = v
            except KeyError:
                pass

    X_df = X_df.interpolate(method="linear", limit_direction="both", axis=0)
    X_df = X_df.bfill().ffill().fillna(0.0)
    return X_df.values.astype(np.float32), use_kps  # (T,6)

def zscore_per_file(X: np.ndarray, eps=1e-6) -> np.ndarray:
    mu = X.mean(axis=0, keepdims=True)
    sd = X.std(axis=0, keepdims=True)
    return (X - mu) / (sd + eps)

def normalize_tailset_minmax(X: np.ndarray, used_kps: list[str], ref_name="tail set", eps=1e-6) -> np.ndarray:
    low = [s.lower() for s in used_kps]
    if ref_name.lower() not in low:
        raise ValueError(f"'{ref_name}' が used_kps にありません: {used_kps}")
    r = low.index(ref_name.lower())

    Xc = X.copy()
    cx, cy = X[:, 2*r], X[:, 2*r+1]
    for i in range(len(used_kps)):
        Xc[:, 2*i]   -= cx
        Xc[:, 2*i+1] -= cy

    mn = Xc.min(axis=0, keepdims=True)
    mx = Xc.max(axis=0, keepdims=True)
    return (Xc - mn) / (mx - mn + eps)

def make_windows(X: np.ndarray, seq_len: int, stride: int):
    n = X.shape[0]
    if n < seq_len:
        return np.empty((0, seq_len, X.shape[1]), dtype=X.dtype), []
    starts = list(range(0, n - seq_len + 1, stride))
    Xw = np.stack([X[s:s+seq_len] for s in starts], axis=0)
    return Xw, starts

def build_dataset(csv_paths, seq_len=SEQ_LEN, stride=STRIDE):
    Xs, ys, fids, starts_all = [], [], [], []
    used_kps_any = None
    for p in csv_paths:
        y = infer_label_from_filename(p)
        X_raw, used_kps = read_dlc_3kp_xy(p, KEYPOINTS, USE_LIKELIHOOD, MIN_KEEP_LIKELIHOOD)
        if used_kps_any is None:
            used_kps_any = used_kps
        if X_raw.shape[1] != DIMS:
            raise ValueError(f"{os.path.basename(p)}: 次元{X_raw.shape[1]} != 期待{DIMS}")

        if NORMALIZE_MODE == "zscore":
            Xn = zscore_per_file(X_raw)
        elif NORMALIZE_MODE == "tail_minmax":
            Xn = normalize_tailset_minmax(X_raw, used_kps)
        else:
            raise ValueError("NORMALIZE_MODE は 'zscore' or 'tail_minmax'")

        Xw, sidx = make_windows(Xn, seq_len, stride)
        if Xw.shape[0] == 0:
            print(f"[WARN] {os.path.basename(p)}: フレーム不足でスキップ")
            continue

        Xs.append(Xw)
        ys.append(np.full((Xw.shape[0],), y, dtype=np.int64))
        fids.extend([os.path.basename(p)]*Xw.shape[0])
        starts_all.extend(sidx)

    if not Xs:
        raise RuntimeError("データが作れませんでした。CSVと命名（ivdd*/normal*）を確認してください。")
    X = np.concatenate(Xs, axis=0)
    y = np.concatenate(ys, axis=0)
    fids = np.array(fids)
    starts_all = np.array(starts_all)
    print(f"[INFO] 使用キーポイント: {used_kps_any}")
    return X, y, fids, starts_all

def build_model(seq_len: int, dims: int, n_hidden: int, l2_lambda: float=1e-4) -> keras.Model:
    reg = keras.regularizers.l2(l2_lambda)
    inp = keras.Input(shape=(seq_len, dims))
    x   = keras.layers.TimeDistributed(keras.layers.Dense(n_hidden, activation="relu", kernel_regularizer=reg))(inp)
    x   = keras.layers.LSTM(n_hidden, return_sequences=True, kernel_regularizer=reg)(x)
    x   = keras.layers.LSTM(n_hidden, kernel_regularizer=reg)(x)
    out = keras.layers.Dense(1, kernel_regularizer=reg)(x)  # logits
    return keras.Model(inp, out, name="ivdd_lstm_3kp")

# ====== データ読み込み ======
csv_files = sorted(glob.glob(TRAIN_GLOB))
if not csv_files:
    raise FileNotFoundError(f"学習CSVが見つかりません: {TRAIN_GLOB}")

X, y, file_ids, starts = build_dataset(csv_files, SEQ_LEN, STRIDE)
print("X:", X.shape, "y:", y.shape, "files:", len(np.unique(file_ids)))

# ====== 分割 ======
if VAL_SPLIT_BY_FILE:
    uniq = np.unique(file_ids)
    tr_files, va_files = train_test_split(uniq, test_size=0.2, random_state=42, shuffle=True)
    tr_mask = np.isin(file_ids, tr_files)
    va_mask = np.isin(file_ids, va_files)
    X_train, y_train, starts_tr = X[tr_mask], y[tr_mask], starts[tr_mask]
    X_val,   y_val,   starts_va = X[va_mask], y[va_mask], starts[va_mask]
    file_ids_tr, file_ids_va = file_ids[tr_mask], file_ids[va_mask]
else:
    X_train, X_val, y_train, y_val, starts_tr, starts_va, file_ids_tr, file_ids_va = train_test_split(
        X, y, starts, file_ids, test_size=0.2, random_state=42, stratify=y
    )

print("train:", X_train.shape, "val:", X_val.shape)

# ====== class_weight ======
if set(np.unique(y_train)) == {0,1}:
    cw = compute_class_weight("balanced", classes=np.array([0,1]), y=y_train)
    class_weight = {0: float(cw[0]), 1: float(cw[1])}
else:
    class_weight = None
print("class_weight:", class_weight)

# ====== モデル ======
model = build_model(SEQ_LEN, DIMS, N_HIDDEN, L2_LAMBDA)
opt = keras.optimizers.Adam(learning_rate=LR)
model.compile(optimizer=opt, loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy(name="accuracy")])
model.summary()

best_path  = os.path.join(MODEL_DIR, f"ivdd_lstm_3kp_{RUN_ID}_best.keras")
final_path = os.path.join(MODEL_DIR, f"ivdd_lstm_3kp_{RUN_ID}_final.keras")

cbs = [
    keras.callbacks.ModelCheckpoint(best_path, monitor="val_accuracy", mode="max",
                                    save_best_only=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", mode="min",
                                      factor=0.5, patience=5, min_lr=1e-5, verbose=1),
]

hist = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS, batch_size=BATCH_SIZE,
    class_weight=class_weight, callbacks=cbs, verbose=1
)

# ====== 学習曲線（1枚） ======
curve_png = os.path.join(FIG_DIR, f"curve_{RUN_ID}.png")
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.plot(hist.history["loss"]); plt.plot(hist.history["val_loss"]); plt.title("Loss"); plt.legend(["train","val"])
plt.subplot(1,2,2); plt.plot(hist.history["accuracy"]); plt.plot(hist.history["val_accuracy"]); plt.title("Accuracy"); plt.legend(["train","val"])
plt.tight_layout(); plt.savefig(curve_png, dpi=150); plt.close()
print("[INFO] saved curve:", curve_png)

# ====== 検証(ウィンドウ)＆誤分類CSV ======
logits_val = model.predict(X_val, batch_size=64)
p_ivdd = tf.math.sigmoid(logits_val).numpy().ravel()
y_pred = (p_ivdd >= 0.5).astype(int)

print("\n[Window-level] classification_report:")
print(classification_report(y_val, y_pred, target_names=CLASS_NAMES, digits=4))
print("[Window-level] confusion matrix:\n", confusion_matrix(y_val, y_pred))

# 誤分類ウィンドウの保存
miss_mask = (y_val != y_pred)
df_miss = pd.DataFrame({
    "file": file_ids_va[miss_mask],
    "start": starts_va[miss_mask],
    "true": [CLASS_NAMES[t] for t in y_val[miss_mask]],
    "pred": [CLASS_NAMES[p] for p in y_pred[miss_mask]],
    "p_ivdd": p_ivdd[miss_mask],
    "p_normal": 1.0 - p_ivdd[miss_mask],
})
miss_csv = os.path.join(VALERR_DIR, f"val_misclassified_{RUN_ID}.csv")
df_miss.to_csv(miss_csv, index=False, encoding="utf-8-sig")
print("[INFO] saved misclassified csv:", miss_csv)

# ====== 最終保存 ======
model.save(final_path)
print("[INFO] saved model:", final_path)
print("[DONE] 3KP training complete:", NORMALIZE_MODE)


TF: 2.19.0


AttributeError: module 'numpy.random' has no attribute 'set_seed'